# 1. Import Libraries

In [77]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency, f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support 

In [35]:
import warnings
warnings.filterwarnings('ignore')

# 2. Load Datasets

In [36]:
a1 = pd.read_excel('case_study1.xlsx')
a2 = pd.read_excel('case_study2.xlsx')

In [37]:
a1.head()
# secured_TL => KUCH girvi rakh kar loan lena

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [38]:
a2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [39]:
df1 = a1.copy()
df2 = a2.copy()

In [40]:
print(df1.shape)
print(df2.shape)

(51336, 26)
(51336, 62)


# 3. Data Cleaning

- after visualization of df1 using ms excel we figure out only Age_Oldest_TL and Age_Newest_TL have only 40 null value(-9999)

In [41]:
# imputation(dataset is to large so we remove the 40 rows who has -999999)

df1 = df1.loc[df1['Age_Oldest_TL']!=-99999]
df1.shape

(51296, 26)

- After visualization of df2 using ms excel we figure out some of the column have more than 10000 0f the null value which 20% of the whole data.

In [42]:
# imputation or feature selection

columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i]==-99999].shape[0]>10000:
        columns_to_be_removed.append(i)

In [43]:
df2 = df2.drop(columns_to_be_removed, axis=1)
df2.shape

(51336, 54)

In [44]:
df2

,PROSPECTID,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,11,29,0,0,0,0,0,21,...,0.0,0.0,0.0,0.0,1,0,PL,PL,696,P2
1,2,47,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,9,25,1,9,8,0,0,10,...,0.0,0.0,0.0,0.0,1,0,ConsumerLoan,others,693,P2
3,4,-99999,0,0,0,0,0,0,0,5,...,0.0,0.0,0.0,0.0,0,0,others,others,673,P2
4,5,583,0,0,0,0,0,0,0,53,...,0.0,0.0,0.0,0.0,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15,2,24,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57,0,0,0,0,0,0,0,6,...,0.0,0.0,0.0,0.0,0,0,others,others,702,P1
51333,51334,32,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0,0,ConsumerLoan,others,661,P3
51334,51335,58,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,others,686,P2


- time_since_recent_payment is only column that have -99999 approximatly 40000 then we remove all the row that have -99999
  

In [45]:
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [46]:
df2.shape

(42066, 54)

## 3.1 create new df

In [47]:
# finding the common column to concatenate df1 and df2
for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [48]:
# Merge the two dataframes, using innerjoin two remove any type of all null values that are present

df = pd.merge(df1, df2, how='inner', left_on=['PROSPECTID'], right_on=['PROSPECTID'])
df.shape

(42064, 79)

In [49]:
df.isna().sum().sum()

0

## 3.2 Sepration of Features

### Categorical Features

In [50]:
cat_list = []
for i in df.columns:
    if df[i].dtype=='object':
        cat_list.append(i)

cat_list

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [51]:
df['MARITALSTATUS'].value_counts()

MARITALSTATUS
Married    30886
Single     11178
Name: count, dtype: int64

In [52]:
df['EDUCATION'].value_counts()

EDUCATION
GRADUATE          14140
12TH              11703
SSC                7241
UNDER GRADUATE     4572
OTHERS             2291
POST-GRADUATE      1898
PROFESSIONAL        219
Name: count, dtype: int64

In [53]:
df['last_prod_enq2'].value_counts()

last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64

#### chi_sq test (feature selection)

In [54]:
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2, p_val, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i , '---', p_val)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


- we reject the null hypothesis(not associated). means all are associated with target value. becoze p_val < 0.05

### Numerical Features

In [55]:
numerical_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
        numerical_columns.append(i)

In [56]:
# numerical_columns

#### **check Multi-Collinearity**
- check the features are associated or not
- Multicolilinearity v/s Correlation
  - correlation is specific to linear relationship b/w columns. In case of convex function, correlation gives misleading values
  - while multicollinearity is Predictability of each features by other features
- for multicollinearity we check vif value. in case of banking domain vif<=6 then multicolinearity is not occure, otherwise multicollinearity occure. otherwise vif<=5

In [57]:
vif_data = df[numerical_columns]
total_columns = vif_data.shape[1]
column_to_be_selected =[]
column_index = 0

- Here we check vif sequentially. lets if vif_data have 10 columns, then in vif 1st is regressed with remaining 9 ftrs. lets assume in case of second vif value is greater than 6 then we immediately drop it that doesn't take part in calculating vif value for other remaining features. this process reduce time complexity and increase performance of model
  - ex: v1, v2, v3, v4, v5, v6, v7, v8, v9 v10 are 10 ftrs. lets v2, v3, v5 => multicollinear then we only remove 2 out of 3 to remove multicollinearity.    

In [58]:

for i in range(0, total_columns):
    vif_value = variance_inflation_factor(vif_data, column_index)
    print(column_index, '----', vif_value)

    if vif_value <= 6:
        column_to_be_selected.append(numerical_columns[i])
        column_index = column_index + 1
    else:
        vif_data = vif_data.drop([numerical_columns[i]], axis=1)

0 ---- inf
0 ---- inf
0 ---- 11.320180023967996
0 ---- 8.363698035000336
0 ---- 6.520647877790933
0 ---- 5.149501618212625
1 ---- 2.611111040579735
2 ---- inf
2 ---- 1788.7926256209232
2 ---- 8.601028256477228
2 ---- 3.8328007921530785
3 ---- 6.099653381646739
3 ---- 5.5813520096427585
4 ---- 1.985584353098778
5 ---- inf
5 ---- 4.809538302819343
6 ---- 23.270628983464636
6 ---- 30.595522588100053
6 ---- 4.3843464059655854
7 ---- 3.0646584155234238
8 ---- 2.898639771299253
9 ---- 4.377876915347322
10 ---- 2.2078535836958433
11 ---- 4.916914200506864
12 ---- 5.214702030064725
13 ---- 3.3861625024231476
14 ---- 7.840583309478997
14 ---- 5.255034641721438
15 ---- inf
15 ---- 7.380634506427238
15 ---- 1.4210050015175733
16 ---- 8.083255010190323
16 ---- 1.6241227524040112
17 ---- 7.257811920140003
17 ---- 15.59624383268298
17 ---- 1.8258570471324314
18 ---- 1.5080839450032666
19 ---- 2.172088834824577
20 ---- 2.623397553527229
21 ---- 2.2959970812106167
22 ---- 7.360578319196446
22 ---- 2.1

In [59]:
len(column_to_be_selected)

39

In [67]:
# pd.DataFrame(column_to_be_selected)

,0
0,pct_tl_open_L6M
1,pct_tl_closed_L6M
2,Tot_TL_closed_L12M
3,pct_tl_closed_L12M
4,Tot_Missed_Pmnt
5,CC_TL
6,Home_TL
7,PL_TL
8,Secured_TL
9,Unsecured_TL


In [78]:
selected_numerical_columns = []

for i in column_to_be_selected:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a,b) if group=='P1']
    group_P2 = [value for value, group in zip(a,b) if group=='P2']
    group_P3 = [value for value, group in zip(a,b) if group=='P3']
    group_P4 = [value for value, group in zip(a,b) if group=='P4']

    f_stats, p_val = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_val <= 0.05:
        selected_numerical_columns.append(i)

In [81]:
len(selected_numerical_columns)

37

- finally we select total 42 features(37 numerical and 5 categorical)

In [82]:
# concattinating all the selected feacture

features = selected_numerical_columns + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
new_df = df[features + ['Approved_Flag']]

In [85]:
new_df

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,0.000,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,0.000,0.0,0,0,Married,12TH,M,ConsumerLoan,ConsumerLoan,P4
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,0.000,0.0,0,0,Married,SSC,M,others,others,P1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,1.000,0.0,0,0,Married,SSC,M,ConsumerLoan,others,P3
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,0.000,0.0,0,0,Single,UNDER GRADUATE,F,ConsumerLoan,others,P2


In [88]:
new_df.to_csv('cleaned_data.csv', index=False)